# Redes Neurais Para Finanças
#### by Maycon Cypriano Batestin

# Dicionário


Fields	                                                  | Type  	  |    Description                              |
----------------------------------------------------------|:---------:|:-------------------------------------------:|
Date 	  										  	  |string     | Data da alteração |
Open														  |float    | preço da abertura                        |
High		     										  |float     | preço mais alto no dia	               |
Low | float | preco mais baixo no dia
Close | float | preco de fechamento
Adj Close | float | preco de fechamento, com ajustes de distribuições de dividendos e/ou ganhos de capital.
Volume | float | Volume total do dia
  

# Instalando os pacotes

In [ ]:
!pip install pandas numpy scikit-learn keras matplotlib plotly yfinance

# Documentação

1. ** Pandas ** -> [Link](https://pandas.pydata.org/docs/)
2. ** Numpy ** -> [Link](https://numpy.org/doc/)
4. ** Scikit Learn ** -> [Link](https://scikit-learn.org/stable/)
5. ** Keras ** -> [Link](https://keras.io/api/)
6. ** Tensor Flow ** -> [Tensor Flow](https://www.tensorflow.org/api_docs/python/tf/keras)

7. ** yfinance ** >[Link](https://pypi.org/project/yfinance/)

8. ** Plotly ** > [Link](https://plotly.com/python/statistical-charts/)

# Etapas

- **Coletar Dados**
  - Recolher dados brutos de várias fontes
- **Analisar os Dados**
  - Explorar os dados para identificar padrões e anomalias
- **Preparar os Dados**
  - Limpar e transformar os dados para torná-los utilizáveis
- **Treinar os Dados**
  - Usar os dados preparados para treinar o modelo de machine learning
- **Testar os Dados**
  - Testar o modelo treinado com um conjunto de dados de teste
- **Avaliar o Modelo**
  - Avaliar o desempenho do modelo e ajustar conforme necessário


# Obtendo o dataset

In [ ]:
import pandas as pd
import yfinance as yf
import os

folder = 'datasets'
if not os.path.exists(folder):
  os.makedirs(folder)


name = "NVDA"
file = os.path.join(folder, f'{name}.csv')
data = yf.download(name, start='2004-01-01', end='2024-06-19')
data.to_csv(file)
df = pd.read_csv(file)
df

# Análise Exploratória

In [ ]:
df.head(), df.tail()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df[df['Open'] <= 0.080583]

In [ ]:
df_corr = df[["Open", "High", "Low", "Close", "Adj Close", "Volume"]]
df_corr.corr()

In [ ]:
import plotly.express as px

fig = px.imshow(df_corr.corr())
fig.show()

In [ ]:
corr_matrix = pd.DataFrame(df_corr.corr())

fig = px.imshow(corr_matrix, x=corr_matrix.columns, y=corr_matrix.columns, zmin = 1, zmax=1, text_auto=True)
fig.update_layout(title='Matriz de Correlação', yaxis_title=" ", xaxis_title=" ")
fig.show()

In [ ]:
fig = px.line(df, x='Date', y='Close', title=f'Fechamento do preço das ações de {name}')
fig.show()

# Preparando os Dados

In [ ]:
px.box(df, y='Close')

In [ ]:
Q1 = df['Close'].quantile(0.25) #primeiro quartil
Q3 = df['Close'].quantile(0.75) #terceiro quartil
IQR = Q3 - Q1 #intervalo interquartil
outliers = df[(df['Close'] < (Q1 - 1.5 * IQR)) | (df['Close'] > (Q3 + 1.5 * IQR))]
outliers

In [ ]:
media_close = df['Close'].median()

for outlier in outliers:
  df['Close'].replace(outlier, media_close, inplace=True)

df

In [ ]:
new_data = df[df['Close'].isin(outliers)]
px.box(df, y='Close')

# Preparando os Dados

In [ ]:
df['Date'].unique().max()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date'].unique().max()

In [ ]:
total_rows = len(df)
legs = 21

df_train = df.iloc[:total_rows - legs]
df_test = df.iloc[total_rows - legs:]

print(len(df))
print(len(df_train))
print(len(df_test))
print(len(df_train) + len(df_test))


In [ ]:
training_set = df_train['Close'].values
training_set

In [ ]:
#processamento normalizacao dos dados
#processo de escalamento dos dados

from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set.reshape(-1,1))
training_set_scaled

In [ ]:
intervalo_entre_os_dias = 60


In [ ]:

X_train = []
y_train = []

for i in range(intervalo_entre_os_dias, len(training_set)):
  X_train.append(training_set_scaled[i-intervalo_entre_os_dias:i, 0]) #vai pegar sempre os 60 dias para tras
  y_train.append(training_set_scaled[i, 0]) #vai pegar sempre todos os valores do intervalo

import numpy as np

X_train, y_train = np.array(X_train), np.array(y_train)


In [ ]:
X_train.shape

#  Treino


In [ ]:
batch_size = len(X_train)
time_steps = intervalo_entre_os_dias
features = 1

X_train = np.reshape(X_train, (batch_size, time_steps, features))

In [ ]:
X_train.shape

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

rnn = Sequential()


In [ ]:
rnn.add(LSTM(units=50, return_sequences=True, input_shape=(time_steps, features)))
rnn.add(Dropout(0.5))

rnn.add(LSTM(units=50, return_sequences=True))
rnn.add(Dropout(0.5))

rnn.add(LSTM(units=50, return_sequences=True))
rnn.add(Dropout(0.5))

rnn.add(LSTM(units=50))
rnn.add(Dropout(0.5))

rnn.add(Dense(units=1))

In [ ]:
from tensorflow.keras.utils import plot_model

plot_model(rnn, to_file='redes_graficas.png', show_shapes=True,  show_layer_names=True, expand_nested=True)


In [ ]:
rnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
rnn.fit(X_train, y_train, epochs = 100, batch_size = 32)

# Teste

In [ ]:
# Testando

fechamento_real = df_test['Close'].values

dataset_total = pd.concat((df['Close'], df_test['Close']), axis=0)
inputs = dataset_total[len(dataset_total) - len(df_test) - intervalo_entre_os_dias:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

inputs = sc.transform(inputs)



In [ ]:
X_test = []
for i in range(intervalo_entre_os_dias, len(inputs)):
  X_test.append(inputs[i-intervalo_entre_os_dias:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))



In [ ]:
print(f'previsao oficial: {rnn.predict(X_test)}')
print(f'valor real: {sc.inverse_transform(rnn.predict(X_test))}')

variacao_final = sc.inverse_transform(rnn.predict(X_test))
print(variacao_final)

# Avaliação de Resultados

In [ ]:

Import matplotlib as plt

plt.plot(fechamento_real, color = 'red', label = f'Dados Reais de Ações da {name}')
plt.plot(variavel_final, color = 'blue', label = f'Dados Previstos de Ações do {name}')
plt.title('Previsão de Preços de Ações')
plt.xlabel('Tempo')
plt.ylabel('Preços de Ações')
plt.legend()
plt.show()